In [43]:
# import libraries (you may add additional imports but you may not have to)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighborsy

In [44]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-07-22 07:14:24--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M  22.6MB/s    in 1.1s    

2025-07-22 07:14:25 (22.6 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Book-Ratings.csv     
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Books.csv            
replace BX-Users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Users.csv            


In [45]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [46]:
# clean data set: remove users with less than 200 ratings and books with less than 100 ratings.
common_users = df_ratings['user'].value_counts()[lambda x: x >= 200].index
common_books = df_ratings['isbn'].value_counts()[lambda x: x >= 100].index

df_ratings = df_ratings[df_ratings['user'].isin(common_users) & df_ratings['isbn'].isin(common_books)]

In [47]:
user_item_matrix = df_ratings.pivot_table(index='user', columns='isbn', values='rating')
matrix_filled = user_item_matrix.fillna(0)

model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(matrix_filled.T)

NearestNeighbors(algorithm='brute', metric='cosine')

In [48]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  query_isbn = df_books.loc[df_books['title'] == book, 'isbn'].values[0]
  distances, indices = model.kneighbors([matrix_filled.T.loc[query_isbn]], n_neighbors=6)

  # remove the book itself and sort like required
  distances = sorted(distances[0][1:], reverse=True)
  indices = sorted(indices[0][1:], reverse=True)

  # build return object
  recommended_books = [book, []]
  for dist, index in zip(distances, indices):
    isbn = matrix_filled.T.index[index]
    title = df_books.loc[df_books['isbn'] == isbn, 'title'].values[0]
    recommended_books[1].append([title, float(dist)])

  return recommended_books

In [49]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016210794448853], ['The Surgeon', 0.7708583474159241], ['The Weight of Water', 0.7699410915374756], ['The Lovely Bones: A Novel', 0.7677075266838074], ['I Know This Much Is True', 0.7234864234924316]]]
You passed the challenge! 🎉🎉🎉🎉🎉
